BITTE FENSTER NICHT SCHLIESSEN!

PLEASE DO NOT CLOSE!!

#####Additional Sensor Logging########

18.01.2022
Roman

In [19]:
HOST = "localhost"
PORT = 4223
uids = {'LB_SWK':'NHT','LB_Roof_insulated':'NH2','LB_Roof_uninsulated':'NR8'}

from tinkerforge.ip_connection import IPConnection
from tinkerforge.bricklet_humidity_v2 import BrickletHumidityV2
import pandas as pd
import datetime as dt
import time
import os
import logging
import numpy as np

In [23]:
logging.basicConfig(
        format='%(asctime)s -- %(levelname)s -- %(message)s', 
        datefmt='%d.%m.%Y %H:%M:%S', 
        level=logging.INFO,
        #encoding='utf-8',
        handlers=[
            logging.FileHandler("LB_addtional.log"),
            logging.StreamHandler()]
    )
today = dt.date.today()
workingdir = f'E:/Data-Sync/Dropbox/1_rohdaten/LB_additional'
logname = f'knollerlogger_{str(today)}.csv'

current_logfile = os.path.join(workingdir, logname)

if os.path.isdir(workingdir) == False:
    os.makedirs(workingdir)

In [24]:
try:
    df = pd.read_csv(current_logfile,
                     parse_dates=['Datetime'],
                     infer_datetime_format=True,
                     index_col='Datetime'
                     )
    logging.info('database loaded')
except FileNotFoundError as e:
    logging.warning(e)
    logging.info(f'new logfile will be created...')
    df = pd.DataFrame()

18.01.2022 09:28:42 -- WARNING -- [Errno 2] File E:/Data-Sync/Dropbox/1_rohdaten/MH_additional\knollerlogger_2022-01-18.csv does not exist: 'E:/Data-Sync/Dropbox/1_rohdaten/MH_additional\\knollerlogger_2022-01-18.csv'
18.01.2022 09:28:42 -- INFO -- new logfile will be created...


In [25]:
#setup sensors
for id, uid in uids.items():
        try:
            ipcon = IPConnection() # Create IP connection
            h = BrickletHumidityV2(uid, ipcon) # Create device object
            ipcon.connect(HOST, PORT) # Connect to brickd
        except Exception as e:
            logging.warning(e)
            logging.info(f'Verbindung zu UID: {uid} konnte nicht hergestellt werden.')
            continue
        h.set_moving_average_configuration(60,60)
        h.set_status_led_config(0)

In [ ]:
logging.info(f'Aufzeichnung gestartet!')
while True:
    try:
        temp = {}
        today = dt.date.today()
        df = df.loc[today:]
        for id, uid in uids.items():
            try:
                ipcon = IPConnection() # Create IP connection
                h = BrickletHumidityV2(uid, ipcon) # Create device object
                ipcon.connect(HOST, PORT) # Connect to brickd
                temp[f'{id}_Tair'] = h.get_temperature()/100.0
                temp[f'{id}_RH'] =  h.get_humidity()/100.0
                ipcon.disconnect()
            except Exception as e:
                logging.warning(e)
                temp[f'{id}_Tair'] = np.NaN
                temp[f'{id}_RH'] =  np.NaN
                logging.info(f'Verbindung zu UID: {uid} konnte nicht hergestellt werden.')
                continue


        df_temp = pd.DataFrame.from_dict(temp,orient='index',columns=[dt.datetime.now()]).T
        df_temp.index.name = 'Datetime'
        df = df.append(df_temp)
        df.to_csv(current_logfile)
        print(f'{dt.datetime.now()} : {temp}',end='\r')
        time.sleep(60)
    except Exception as e:
        logging.warning(e)

18.01.2022 09:28:44 -- INFO -- Aufzeichnung gestartet!
